In [1]:
from simulation import Lattice, Geometry
from symbolic.generator import LBM
import symbolic.D2Q9 as D2Q9

In [2]:
lbm = LBM(D2Q9)

In [3]:
def cavity(geometry, x, y):
    if x == 1 or y == 1 or x == geometry.size_x-2:
        return 2
    elif y == geometry.size_y-2:
        return 3
    else:
        return 1

In [4]:
boundary = """
    if ( m == 2 ) {
        u_0 = 0.0;
        u_1 = 0.0;
    }
    if ( m == 3 ) {
        u_0 = 0.1;
        u_1 = 0.0;
    }
"""

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy

In [6]:
def generate_moment_plot(lattice, m):
    velocity = numpy.ndarray(shape=tuple(reversed(lattice.geometry.inner_size())))
    for x, y in lattice.geometry.inner_cells():
        velocity[y-1,x-1] = numpy.sqrt(m[1,lattice.gid(x,y)]**2 + m[2,lattice.gid(x,y)]**2)
    plt.figure(figsize=(8,8))
    plt.imshow(velocity, origin='lower', vmin=0.0, cmap=plt.get_cmap('seismic'))

In [7]:
def test(nX=64, nY=64, nSteps=1000, tau=0.6):
    lattice = Lattice(
        descriptor = D2Q9,
        geometry   = Geometry(nX, nY),

        moments = lbm.moments(optimize = False),
        collide = lbm.bgk(f_eq = lbm.equilibrium(), tau = tau),

        boundary_src = boundary)
    lattice.setup_geometry(cavity)
    
    for i in range(0,nSteps):
        lattice.evolve()
    generate_moment_plot(lattice, lattice.get_moments())

In [8]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive

In [9]:
interactive(test, nX=(32,1024,32), nY=(32,1024,32), nSteps=(0,100000,500), tau=(0.515,1.0,0.01))

interactive(children=(IntSlider(value=64, description='nX', max=1024, min=32, step=32), IntSlider(value=64, de…